content
- goal
- problems / examples
  -missing liabilities and lieabilitiesnoncurrent tags
- what does the standardizer to
 - pre steps
   - deduplication
   - correct errors
 - main rules
 - post rule
 - validation
- loading and saving information
- input 
   -> filtered data 
   -> only statement, only one currency, only main company, only standardized tags
- what kind of logs are produced, what do they show
- what you should consider if using standardized information
  -> check logs / check validation summary 
- limitations
  wrong data, missing tags in data
  none standardized tags
  
- Empirical approach, definition is not taken into account